<a href="https://colab.research.google.com/github/sujalsethi30/Sales4castr/blob/main/sales4castr2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y numpy pandas scikit-learn
!pip install numpy==1.24.4 pandas==2.2.2 scikit-learn==1.3.2
!pip install torch==2.2.1 pytorch-lightning==2.2.1
!pip install u8darts[torch]==0.26.0 xgboost==1.7.6


In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 --force-reinstall
!pip install u8darts[all]==0.24.0 --force-reinstall


In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import r2_score, rmse, mae
import matplotlib.pyplot as plt

df = pd.read_csv("/content/sample_data/dataset.csv")
df.columns = ["Month", "Sales"]
df["Month"] = pd.to_datetime(df["Month"])
df.dropna(inplace=True)

series = TimeSeries.from_dataframe(df, time_col="Month", value_cols="Sales")

train, val = series[:-12], series[-12:]


scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)



model = TFTModel(
    input_chunk_length=24,
    output_chunk_length=12,
    hidden_size=16,
    lstm_layers=1,
    dropout=0.1,
    batch_size=16,
    n_epochs=200,
    add_relative_index=True,
    add_encoders=None,
    random_state=42
)



model.fit(train_scaled, verbose=True)


forecast = model.predict(n=12)

forecast = scaler.inverse_transform(forecast)
val_actual = scaler.inverse_transform(val_scaled)

print("📈 R² Score:", r2_score(val_actual, forecast))
print("📉 RMSE:", rmse(val_actual, forecast))
print("📉 MAE:", mae(val_actual, forecast))

series.plot(label="Actual")
forecast.plot(label="Forecast")
plt.legend()
plt.title("TFT Forecast vs Actual")
plt.show()


In [ ]:
!pip install u8darts[torch]==0.26.0 xgboost==1.7.6

In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler


df = pd.read_csv("/content/sample_data/dataset.csv")
df.columns = ["Month", "Sales"]
df["Month"] = pd.date_range(start="1964-01", periods=len(df), freq="MS")
df.set_index("Month", inplace=True)

series = TimeSeries.from_dataframe(df, value_cols="Sales")
train, val = series[:-12], series[-24:]
scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)


In [ ]:
from darts.models import TFTModel
from darts.metrics import r2_score, rmse, mae

param_sets = [
    {"hidden_size": 16, "dropout": 0.1},
    {"hidden_size": 32, "dropout": 0.2},
    {"hidden_size": 64, "dropout": 0.3},
]

results_tft = []

for params in param_sets:
    model = TFTModel(
    input_chunk_length=24,
    output_chunk_length=12,
    hidden_size=params["hidden_size"],
    dropout=params["dropout"],
    lstm_layers=1,
    n_epochs=150,
    batch_size=8,
    random_state=42,
    add_relative_index=True
)
    model.fit(train_scaled, verbose=False)
    forecast = model.predict(n=12)
    actual = val_scaled[-12:]

    results_tft.append({
        "params": params,
        "R2": r2_score(actual, forecast),
        "RMSE": rmse(actual, forecast),
        "MAE": mae(actual, forecast)
    })


In [ ]:


print("\n🤖 TFT Tuning Results:")
for res in results_tft:
    print(f"Params: {res['params']}")
    print(f"  R²: {res['R2']:.4f}, RMSE: {res['RMSE']:.2f}, MAE: {res['MAE']:.2f}\n")


In [ ]:
import matplotlib.pyplot as plt
from darts.metrics import r2_score, rmse, mae

best_model_idx = 1
best_params = {"hidden_size": 32, "dropout": 0.2}

from darts.models import TFTModel

best_model = TFTModel(
    input_chunk_length=24,
    output_chunk_length=12,
    hidden_size=best_params["hidden_size"],
    dropout=best_params["dropout"],
    lstm_layers=1,
    n_epochs=150,
    batch_size=8,
    random_state=42,
    add_relative_index=True,
)

best_model.fit(train_scaled, verbose=False)
forecast = best_model.predict(n=12)
actual = val_scaled[-12:]


plt.figure(figsize=(10, 5))
actual.plot(label="Actual", lw=2)
forecast.plot(label="Forecast (TFT)", lw=2)
plt.title("Actual vs Forecasted Values (Best TFT Model)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
forecast_original = scaler.inverse_transform(forecast)
actual_original = scaler.inverse_transform(actual)

forecast_values = forecast_original.values()
actual_values = actual_original.values()


from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse_original = np.sqrt(mean_squared_error(actual_values, forecast_values))
mae_original = mean_absolute_error(actual_values, forecast_values)

print(f"Original RMSE: {rmse_original:.2f}")
print(f"Original MAE: {mae_original:.2f}")


In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
import matplotlib.pyplot as plt


df = pd.read_csv("/content/sample_data/dataset.csv")
df.columns = ["Month", "Sales"]
df["Month"] = pd.to_datetime(df["Month"])
df.dropna(inplace=True)


series = TimeSeries.from_dataframe(df, time_col="Month", value_cols="Sales")

train = series[:-12]
val = series[-12:]

scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)

best_params = {"hidden_size": 32, "dropout": 0.2}

model = TFTModel(
    input_chunk_length=24,
    output_chunk_length=12,
    hidden_size=best_params["hidden_size"],
    dropout=best_params["dropout"],
    lstm_layers=1,
    n_epochs=150,
    batch_size=8,
    random_state=42,
    add_relative_index=True,
)

model.fit(train_scaled, verbose=True)

forecast = model.predict(n=12)
forecast = scaler.inverse_transform(forecast)

plt.figure(figsize=(10, 5))
train.plot(label="Train", lw=2)
val.plot(label="Actual", lw=2)
forecast.plot(label="Forecast", lw=2, color='cyan')
plt.title("Sales Forecast using Fine-Tuned TFT Model")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print("Future Forecasted Sales:")
print(forecast)


In [ ]:
import pandas as pd
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
import matplotlib.pyplot as plt

df = pd.read_csv("/content/sample_data/dataset.csv")
df.columns = ["Month", "Sales"]
df["Month"] = pd.to_datetime(df["Month"])
df.dropna(inplace=True)

series = TimeSeries.from_dataframe(df, time_col="Month", value_cols="Sales")

train_cutoff = pd.Timestamp("1972-08-01")
train = series.drop_after(train_cutoff)

scaler = Scaler()
train_scaled = scaler.fit_transform(train)

best_model = TFTModel(
    input_chunk_length=24,
    output_chunk_length=13,
    hidden_size=32,
    dropout=0.2,
    lstm_layers=1,
    n_epochs=150,
    batch_size=8,
    random_state=42,
    add_relative_index=True,
)


best_model.fit(train_scaled, verbose=True)


forecast = best_model.predict(n=13)
forecast = scaler.inverse_transform(forecast)


plt.figure(figsize=(10, 5))
series.plot(label="Historical Sales", lw=2)
forecast.plot(label="Forecast (Sept 1972 - Sept 1973)", lw=2, color='orange')
plt.title("Sales Forecast from Sept 1972 to Sept 1973 (TFT Model)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


print("🔮 Forecasted Sales from Sept 1972 to Sept 1973:")
print(forecast.pd_dataframe())


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
from darts.metrics import r2_score, rmse, mae

# Load dataset
df = pd.read_csv("/content/sample_data/index_1.csv")
df.columns = ["Month", "Sales"]
df["Month"] = pd.to_datetime(df["Month"])
df.dropna(inplace=True)

# Convert to TimeSeries
series = TimeSeries.from_dataframe(df, time_col="Month", value_cols="Sales")

# Train on first 11 months, validate on last 1
train, val = series[:-1], series[-1:]

# Scaling
scaler = Scaler()
train_scaled = scaler.fit_transform(train)
val_scaled = scaler.transform(val)

# Define and train TFT model (adjusted output length)
tft_model = TFTModel(
    input_chunk_length=6,      # Smaller due to limited data
    output_chunk_length=1,     # Predict just 1 month
    hidden_size=16,
    dropout=0.2,
    lstm_layers=1,
    n_epochs=150,
    batch_size=4,
    random_state=42,
    add_relative_index=True,
)

tft_model.fit(train_scaled, verbose=True)

# Predict last month
forecast_scaled = tft_model.predict(n=1)
forecast = scaler.inverse_transform(forecast_scaled)
val_actual = scaler.inverse_transform(val_scaled)

# Plot results
plt.figure(figsize=(8, 4))
val_actual.plot(label="Actual Sales", lw=2)
forecast.plot(label="Predicted Sales", lw=2, linestyle="--")
plt.title("TFT - Actual vs Predicted Sales (Last Month)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Metrics
print("📈 R² Score:", r2_score(val_actual, forecast))
print("📉 RMSE:", rmse(val_actual, forecast))
print("📉 MAE:", mae(val_actual, forecast))
